In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import cv2  
from tqdm import tqdm
from random import randint
import json
import matplotlib.image as mpimg
from matplotlib import rcParams
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from tensorflow.keras.applications.vgg16 import preprocess_input,VGG16

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Using TensorFlow backend.


In [2]:
train_path = "data/train"
test_path = "data/test"

In [3]:
train = os.listdir(train_path)
test = os.listdir(test_path)

In [4]:
train_image = [cv2.resize(cv2.cvtColor(cv2.imread(os.path.join(train_path, image)), cv2.COLOR_BGR2RGB),(150,150))  for image in train ]

In [5]:
test_image = [cv2.resize(cv2.cvtColor(cv2.imread(os.path.join(test_path, image)), cv2.COLOR_BGR2RGB),(150,150))  for image in test ]

In [6]:
label = pd.read_csv("data/train_features.csv")
label[["x","y"]]

,x,y
0,-9.380678,3.58272
1,-9.380678,3.58272
2,-9.380678,3.58272
3,-9.380678,3.58272
4,-9.380678,3.58272
...,...,...
7495,-4.680678,35.18272
7496,-4.680678,35.18272
7497,-4.680678,35.18272
7498,-4.680678,35.18272


In [7]:
y_label = []
for index, row in label.iterrows():
    y_label.append((row["x"],row["y"]))
y_label = np.array(y_label)
train_image = np.array(train_image)

In [8]:
test_image = np.array(test_image)

In [9]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [10]:
#vgg16 = VGG16(weights='imagenet',input_shape=[150,150,3],include_top=False)
vgg19 = VGG19(weights='imagenet',input_shape=[150,150,3],include_top=False)
#resnet50 = ResNet50(weights='imagenet',input_shape=[150,150,3],include_top=False)

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers.merge import concatenate
from tensorflow.keras.models import Model

In [12]:
def train_model_full(pre_train):
  for layer in pre_train.layers:
    layer.trainable = False
  layer = layers.Conv2D(filters = 32,kernel_size = (1,1),padding = "same",activation = "relu")(pre_train.layers[-1].output)
  layer = layers.MaxPooling2D(pool_size=(2,2), strides=2,padding="same")(layer)
  layer = layers.Conv2D(filters = 64,kernel_size = (3,3),padding = "same",activation = "relu")(layer)
  layer = layers.Flatten()(layer)
  layer = layers.Dense(1024,activation="relu")(layer)
  layer = layers.Dense(2,activation="linear")(layer)

  model = tf.keras.Model(inputs = pre_train.inputs,outputs = layer)

  return model

In [13]:
opt = keras.optimizers.Adam(learning_rate=0.001)


In [14]:
model_vgg19_full = train_model_full(vgg19)

model_vgg19_full.compile(optimizer=opt,
            loss=tf.keras.losses.MeanAbsoluteError())

In [15]:
model_vgg19_full.fit(train_image, label[["x","y"]], epochs=50, batch_size=128)

Epoch 1/50
59/59 [==============================] - 38s 433ms/step - loss: 33.9394
Epoch 2/50
59/59 [==============================] - 20s 337ms/step - loss: 28.2647
Epoch 3/50
59/59 [==============================] - 21s 353ms/step - loss: 25.8296
Epoch 4/50
59/59 [==============================] - 21s 364ms/step - loss: 24.1265
Epoch 5/50
59/59 [==============================] - 26s 435ms/step - loss: 22.4912
Epoch 6/50
59/59 [==============================] - 27s 463ms/step - loss: 21.4170
Epoch 7/50
59/59 [==============================] - 28s 466ms/step - loss: 20.3369
Epoch 8/50
59/59 [==============================] - 28s 469ms/step - loss: 19.4545
Epoch 9/50
59/59 [==============================] - 28s 469ms/step - loss: 18.5136
Epoch 10/50
59/59 [==============================] - 28s 474ms/step - loss: 17.7874
Epoch 11/50
59/59 [==============================] - 28s 477ms/step - loss: 17.1333
Epoch 12/50
59/59 [==============================] - 28s 476ms/step - loss: 16.5528
E

In [16]:
result = model_vgg19_full.predict(test_image)


In [ ]:
x = []
y = []
for i in result:
   x.append(i[0])
   y.append(i[1]) 

In [ ]:
test_result = pd.read_csv("data/imagenames.csv")

In [ ]:
test_result["x"] = x
test_result["y"] = y

In [ ]:
test_result.to_csv("result/CNN_baseline_vgg19.csv",index=False)